## **Lectura 2: Implementación simplex de transporte** 

1. Balancear el problema:

In [47]:
import numpy as np

In [48]:
def get_balanced_tp(supply, demand, costs, penalties=None):
    # Calculamos el suministro y la demanda total.
    total_supply = sum(supply)
    total_demand = sum(demand)

    # Si el suministro total es menor que la demanda total:
    if total_supply < total_demand:
        # Si no se proporcionan penalizaciones, lanzamos una excepción.
        if penalties is None:
            raise Exception('Supply less than demand, penalties required')
        
        # Creamos nuevas listas de suministro y costos que incluyen la diferencia entre demanda y suministro,
        # usando las penalizaciones proporcionadas para los costos adicionales.
        new_supply = supply + [total_demand - total_supply]
        new_costs = costs + [penalties]
        
        # Devolvemos las nuevas listas de suministro, demanda y costos.
        return new_supply, demand, new_costs
    
    # Si el suministro total es mayor que la demanda total:
    if total_supply > total_demand:
        # Creamos una nueva lista de demanda que incluye la diferencia entre suministro y demanda,
        # y añadimos una fila de ceros a la matriz de costos para los costos adicionales.
        new_demand = demand + [total_supply - total_demand]
        new_costs = costs + [[0 for _ in demand]]
        
        # Devolvemos las nuevas listas de suministro, demanda y costos.
        return supply, new_demand, new_costs
    
    # Si el suministro total es igual a la demanda total, devolvemos las listas originales de suministro, demanda y costos.
    return supply, demand, costs


In [49]:
# Ejemplo 1: oferta menor que la demanda

supply = [40, 30]
demand = [30, 50]
costs = [
    [3, 4],
    [2, 4]
]
penalties = [3, 1]
get_balanced_tp(supply, demand, costs, penalties)

([40, 30, 10], [30, 50], [[3, 4], [2, 4], [3, 1]])

In [50]:
# Ejemplo 2: oferta mayor que la demanda

supply = [40, 30]
demand = [30, 30]
costs = [
    [3, 4],
    [2, 4]
]
get_balanced_tp(supply, demand, costs, penalties)

([40, 30], [30, 30, 10], [[3, 4], [2, 4], [0, 0]])

2. Solución básica factible inicial:

In [51]:
def north_west_corner(supply, demand):
    # Hacemos copias de las listas de suministro y demanda para evitar modificar las listas originales.
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    
    # Inicializamos los índices para iterar sobre las listas de suministro y demanda.
    i = 0
    j = 0
    
    # Inicializamos una lista para almacenar las asignaciones de suministro a demanda.
    bfs = []
    
    # Continuamos el bucle hasta que el número de asignaciones en 'bfs' sea 
    # igual a la suma de las longitudes de suministro y demanda menos uno.
    while len(bfs) < len(supply) + len(demand) - 1:
        
        # Obtenemos la oferta y demanda actual usando los índices i y j.
        s = supply_copy[i]
        d = demand_copy[j]
        
        # La cantidad asignada es el mínimo entre la oferta y la demanda actuales.
        v = min(s, d)
        
        # Actualizamos las listas de suministro y demanda restando la cantidad asignada.
        supply_copy[i] -= v
        demand_copy[j] -= v
        
        # Agregamos la asignación a la lista 'bfs'.
        bfs.append(((i, j), v))
        
        # Si hemos agotado el suministro del punto de origen actual y aún quedan más puntos de origen,
        # incrementamos el índice i.
        if supply_copy[i] == 0 and i < len(supply) - 1:
            i += 1
        
        # Si hemos satisfecho la demanda del destino actual y aún quedan más destinos,
        # incrementamos el índice j.
        elif demand_copy[j] == 0 and j < len(demand) - 1:
            j += 1
            
    # Devolvemos la lista de asignaciones 'bfs'.
    return bfs


In [52]:
#ejemplo: soluciòn bàsica factible

supply = [30, 70, 50]
demand = [40, 30, 40, 40]
bfs = north_west_corner(supply, demand)
print(bfs)

[((0, 0), 30), ((1, 0), 10), ((1, 1), 30), ((1, 2), 30), ((2, 2), 10), ((2, 3), 40)]


3. Cálculo de las variables duales 

In [53]:
def get_us_and_vs(bfs, costs):
    # Inicializamos las listas 'us' y 'vs' con None y de longitud igual
    # a la cantidad de filas y columnas de la matriz de costos respectivamente.
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    
    # Asignamos 0 a la primera posición de 'us' ya que es un punto de inicio para calcular el resto de los valores.
    us[0] = 0
    
    # Creamos una copia de 'bfs' para modificarla durante el proceso sin afectar la original.
    bfs_copy = bfs.copy()
    
    # Continuamos el bucle hasta que 'bfs_copy' esté vacía.
    while len(bfs_copy) > 0:
        # Iteramos sobre cada elemento en 'bfs_copy'.
        for index, bv in enumerate(bfs_copy):
            # Obtenemos los índices i, j del elemento actual en 'bfs_copy'.
            i, j = bv[0]
            
            # Si tanto 'us[i]' como 'vs[j]' son None, continuamos con la siguiente iteración.
            if us[i] is None and vs[j] is None: continue
                
            # Obtenemos el costo asociado a los índices i, j.
            cost = costs[i][j]
            
            # Si 'us[i]' es None, calculamos su valor.
            # Si no, calculamos el valor de 'vs[j]'.
            if us[i] is None:
                us[i] = cost - vs[j]
            else: 
                vs[j] = cost - us[i]
            
            # Eliminamos el elemento actual de 'bfs_copy' y salimos del bucle for.
            bfs_copy.pop(index)
            break
            
    # Devolvemos las listas 'us' y 'vs'.
    return us, vs


In [54]:
# Ejemplo 2: oferta mayor que la demanda

costs = [
    [ 2, 2, 2, 1],
    [10, 8, 5, 4],
    [ 7, 6, 6, 8]
]
supply = [30, 70, 50]
demand = [40, 30, 40, 40]
bfs = north_west_corner(supply, demand)
us, vs = get_us_and_vs(bfs, costs)
print(us)
print(vs)


[0, 8, 9]
[2, 0, -3, -1]


4. Cálculo de los costos reducidos

In [55]:
def get_ws(bfs, costs, us, vs):
    # Inicializamos una lista 'ws' vacía para almacenar los valores calculados.
    ws = []

    # Iteramos sobre cada fila y cada elemento de la fila en la matriz de costos.
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            # Verificamos si la celda actual (i, j) no está en 'bfs', es decir, si es no básica.
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            
            # Si la celda es no básica, calculamos su valor correspondiente y lo añadimos a 'ws'.
            if non_basic:
                ws.append(((i, j), cost - us[i] - vs[j]))
                
    # Devolvemos la lista 'ws'.
    return ws


In [56]:
ws= get_ws(bfs, costs, us, vs)
print(ws)

[((0, 1), 2), ((0, 2), 5), ((0, 3), 2), ((1, 3), -3), ((2, 0), -4), ((2, 1), -3)]


5. Definir si podemos mejorar la solución

In [57]:
def can_be_improved(ws):
    for p, v in ws:
        if v < 0: return True
    return False

In [58]:
can_be_improved(ws)

True

In [59]:
ws_copy = ws.copy()
ws_copy.sort(key=lambda w: w[1])
print(ws_copy)

[((2, 0), -4), ((1, 3), -3), ((2, 1), -3), ((0, 1), 2), ((0, 3), 2), ((0, 2), 5)]


In [60]:
def get_entering_variable_position(ws):
    # Creamos una copia de la lista 'ws' para no modificar la lista original durante el proceso de ordenamiento.
    ws_copy = ws.copy()
    
    # Ordenamos 'ws_copy' basándonos en el segundo elemento de cada tupla (w[1]).
    # Esto ordena la lista de tuplas basándose en los valores de 'w' en orden ascendente.
    ws_copy.sort(key=lambda w: w[1])
    
    # Devolvemos el primer elemento de la primera tupla en 'ws_copy' ordenado, 
    # que corresponde a la posición de la variable de entrada en la matriz de costos.
    return ws_copy[0][0]

In [61]:
get_entering_variable_position(ws)

(2, 0)

In [62]:
def get_possible_next_nodes(loop, not_visited):
    # Obtener el último nodo en 'loop'.
    last_node = loop[-1]
    
    # Obtener los nodos no visitados en la misma fila que 'last_node'.
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    
    # Obtener los nodos no visitados en la misma columna que 'last_node'.
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    
    # Si 'loop' tiene menos de dos nodos, 
    # retornar todos los nodos no visitados en la misma fila y columna que 'last_node'.
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        # Si 'loop' tiene dos o más nodos, determinar si el último movimiento fue en fila.
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        
        # Si el último movimiento fue en fila, retornar los nodos en la columna.
        # Si el último movimiento fue en columna, retornar los nodos en la fila.
        if row_move:
            return nodes_in_column
        return nodes_in_row


In [63]:
def get_loop(bv_positions, ev_position):
    # Definimos una función interna 'inner', que será una función recursiva.
    def inner(loop):
        # Si 'loop' tiene más de tres nodos, comprobamos si puede ser cerrado.
        # Un 'loop' puede ser cerrado si solo hay un nodo posible para visitar a continuación, 
        # que será 'ev_position'.
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: 
                return loop  # Si el 'loop' puede ser cerrado, lo retornamos.
        
        # Obtenemos los nodos no visitados quitando los nodos ya presentes en 'loop' de 'bv_positions'.
        not_visited = list(set(bv_positions) - set(loop))
        
        # Obtenemos los posibles siguientes nodos usando la función 'get_possible_next_nodes'.
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        
        # Iteramos sobre cada nodo posible y llamamos recursivamente a 'inner' con el nuevo 'loop'.
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])  # Añadimos el nuevo nodo a 'loop' y llamamos a 'inner' de nuevo.
            if new_loop: 
                return new_loop  # Si conseguimos un nuevo 'loop', lo retornamos.
    
    # Iniciamos la función interna 'inner' con 'ev_position' como el primer nodo en 'loop'.
    return inner([ev_position])

    


In [64]:
def loop_pivoting(bfs, loop):
    # Dividimos el 'loop' en dos listas: 'even_cells' y 'odd_cells', donde
    # 'even_cells' contiene las celdas en las posiciones pares del 'loop' y
    # 'odd_cells' contiene las celdas en las posiciones impares del 'loop'.
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    
    # Definimos una función lambda 'get_bv', que obtiene el valor de una celda en 'bfs'
    # dado su posición.
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    
    # Encontramos la posición de salida ordenando 'odd_cells' por su valor en 'bfs' y
    # tomando el primero. Luego obtenemos el valor en esa posición.
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    # Inicializamos la nueva lista 'new_bfs'.
    new_bfs = []
    
    # Iteramos sobre las celdas en 'bfs' excluyendo la posición de salida y añadiendo
    # la primera celda de 'loop' con valor 0.
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        # Si la posición 'p' está en 'even_cells', incrementamos su valor por 'leaving_value'.
        if p in even_cells:
            v += leaving_value
        # Si la posición 'p' está en 'odd_cells', decrementamos su valor por 'leaving_value'.
        elif p in odd_cells:
            v -= leaving_value
        
        # Añadimos la posición y el nuevo valor a 'new_bfs'.
        new_bfs.append((p, v))
    
    # Devolvemos 'new_bfs', que contiene las nuevas celdas básicas después del pivoteo.
    return new_bfs


In [65]:
def transportation_simplex_method(supply, demand, costs, penalties=None):
    # Se balancea el problema de transporte para que la oferta y la demanda total sean iguales.
    balanced_supply, balanced_demand, balanced_costs = get_balanced_tp(
        supply, demand, costs, penalties
    )
    
    # Función interna recursiva que será usada para realizar las iteraciones del método simplex.
    def inner(bfs):
        # Calcula los valores de 'us' y 'vs' usando las variables básicas y los costos balanceados.
        us, vs = get_us_and_vs(bfs, balanced_costs)
        
        # Calcula los valores de 'ws' utilizando las variables básicas, los costos balanceados, 'us' y 'vs'.
        ws = get_ws(bfs, balanced_costs, us, vs)
        
        # Si la solución puede ser mejorada, se encuentra la posición de la variable entrante,
        # se construye el bucle y se realiza el pivoteo.
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))
        
        # Si la solución no puede ser mejorada, se retorna las variables básicas.
        return bfs
    
    # Se inicia el método con una solución inicial obtenida por el método de la esquina noroeste.
    basic_variables = inner(north_west_corner(balanced_supply, balanced_demand))
    
    # Se inicializa la matriz de solución con ceros y se llenan las celdas correspondientes con los valores
    # de las variables básicas.
    solution = np.zeros((len(costs), len(costs[0])))
    for (i, j), v in basic_variables:
        solution[i][j] = v
    
    # Se retorna la matriz de solución.
    return solution


In [44]:
def get_total_cost(costs, solution):
    # Inicializamos la variable 'total_cost' a 0. Esta variable almacenará el costo total calculado.
    total_cost = 0
    
    # Iteramos sobre cada fila y cada elemento de la fila en la matriz de costos.
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            # Para cada celda en la matriz de costos, multiplicamos el costo por el valor correspondiente
            # en la matriz de solución y lo añadimos al 'total_cost'.
            total_cost += cost * solution[i][j]
    
    # Finalmente, retornamos el valor de 'total_cost' calculado.
    return total_cost


In [66]:
costs = [
    [ 2, 2, 2, 1],
    [10, 8, 5, 4],
    [ 7, 6, 6, 8]
]
supply = [30, 70, 50]
demand = [40, 30, 40, 40]
solution = transportation_simplex_method(supply, demand, costs)
print(solution)
print('total cost: ', get_total_cost(costs, solution))

[[30.  0.  0.  0.]
 [ 0.  0. 30. 40.]
 [10. 30. 10.  0.]]
total cost:  680.0
